In [49]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker, declarative_base

from sqlalchemy import (
    Table, 
    Column, 
    String,
    Integer, 
    Float,
    Boolean,
    select,
    text,
    func
)

import sqlalchemy as db

In [50]:
engine = create_engine('sqlite:///census.sqlite')

In [51]:
Base = declarative_base()

#Zad2.1
class StudentsTable(Base):
    __tablename__ = 'students'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    grade = Column(Float)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

#Zad2.2
new_records = [
    StudentsTable(name='John', age=30, grade=4.0),
    StudentsTable(name='James', age=25, grade=3.5),
    StudentsTable(name='Camile', age=27, grade=3.7)
    ]
session.add_all(new_records)
session.commit()

session.close()

In [3]:
inspector = inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['census', 'state_fact']


In [4]:
connection = engine.connect()
print(connection)

In [5]:
metadata = MetaData()
census = Table('census', metadata, autoload_with=engine)
print(repr(census))

Table('census', MetaData(), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)


In [39]:
#Zad1.1
stmt = text('SELECT state FROM census')  
result_proxy = connection.execute(stmt)
results = result_proxy.fetchall()
print(results)

[('Illinois', 'M', 0, 89600, 95012), ('Illinois', 'M', 1, 88445, 91829), ('Illinois', 'M', 2, 88729, 89547), ('Illinois', 'M', 3, 88868, 90037), ('Illinois', 'M', 4, 91947, 91111)]


In [37]:
#Zad1.2
stmt = text("SELECT SUM(pop2008) + SUM(pop2000) AS total_pop FROM census WHERE state = 'New York' OR state = 'Alaska'")  
result_proxy = connection.execute(stmt)
results = result_proxy.fetchall()
print(results)

[(39716961,)]


In [41]:
#Zad1.3
stmt = text("SELECT SUM(pop2008) FROM census WHERE state = 'New York'")  
result_proxy = connection.execute(stmt)
results = result_proxy.fetchall()
print(results)

[(19465159,)]


In [68]:
#Zad2.3
stmt = text("SELECT * FROM students")  
result_proxy = connection.execute(stmt)
results = result_proxy.fetchall()
print(results)

[(1, 'Harry', 30, 4.0), (2, 'James', 25, 3.5), (3, 'Camile', 27, 3.7), (4, 'John', 30, 4.0), (11, 'Michale', 38, 4.0)]


In [54]:
#Zad3.1
stmt = text("INSERT INTO students(name, age, grade) VALUES('Michale', 38, 4.0)") 
result = connection.execute(stmt)
inserted_rows_count = result.rowcount

print("Liczba wstawionych wierszy:", inserted_rows_count)

Liczba wstawionych wierszy: 1


In [55]:
#Zad3.2
stmt = text("SELECT * FROM students WHERE id = 1")  
result_proxy = connection.execute(stmt)
results = result_proxy.fetchall()
print(results)

[(1, 'John', 30, 4.0)]


In [57]:
#Zad3.3
stmt = text("UPDATE students SET name = 'Harry' WHERE id = 1")  
result = connection.execute(stmt)
inserted_rows_count = result.rowcount

print("Liczba zaktualizowanych wierszy:", inserted_rows_count)

Liczba wstawionych wierszy: 1


In [67]:
#Zad3.4
stmt = text("DELETE FROM students WHERE id = 8")  
result = connection.execute(stmt)
inserted_rows_count = result.rowcount

print("Liczba usuniętych wierszy:", inserted_rows_count)

Liczba usuniętych wierszy: 1


In [16]:
query = select(census.c.state, census.c.sex)
output = connection.execute(query)
results = output.fetchall()
print(results[:5])

[('Illinois', 'M'), ('Illinois', 'M'), ('Illinois', 'M'), ('Illinois', 'M'), ('Illinois', 'M')]


In [17]:
print(query)

SELECT census.state, census.sex 
FROM census


In [18]:
stmt = select(
    func.sum(census.c.pop2008)
)
results = connection.execute(stmt).fetchall()
print(results)

[(302876613,)]


In [19]:
stmt = select(
    func.sum(census.c.pop2008)
).where(census.c.sex == 'F')

In [20]:
results = connection.execute(stmt).fetchall()
print(results)

[(153959198,)]
